In [ ]:
import os
from pydub import AudioSegment
from pydub.utils import make_chunks
from vosk import Model, KaldiRecognizer
import sys
import json
import csv
from tqdm import tqdm
import numpy as np

chunks_dir = '/home/mabikana/Documents/PhD/Datasets/RECOLA/audio_chunks'
speech_to_text_model_dir = '/home/mabikana/Documents/PhD/SER Code/EmergencyOutcomePrediction/speech_to_text/model'

speech_to_text_model = Model(speech_to_text_model_dir)

# Large vocabulary free form recognition
rec = KaldiRecognizer(speech_to_text_model, 8000)

In [ ]:
def chunk_to_text(chunk_path):
    wf = open(chunk_path, "rb")
    textResults = []
    results = ""

    while True:
        data = wf.read(8000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            recognizerResult = rec.Result()
            results = results + recognizerResult
            resultDict = json.loads(recognizerResult)
            textResults.append(resultDict.get("text", ""))

    results = results + rec.FinalResult()
    resultDict = json.loads(rec.FinalResult())
    textResults.append(resultDict.get("text", ""))
    return ' '.join(textResults)

In [ ]:
wav_text_data = []
max_text_len = 0
min_text_len = 999999
all_lens = []


files = os.listdir(chunks_dir)
for file in tqdm(files, desc = 'files'):
    text = chunk_to_text(chunks_dir + '/' + file)

    if(len(text) > max_text_len):
        max_text_len = len(text)
    if(len(text) < min_text_len):
        min_text_len = len(text)

    all_lens.append(len(text))

    chunk_row = {'chunk_path': file, 'text': text}
    wav_text_data.append(chunk_row)

print('Max text length = ' + str(max_text_len))
print('Min text length = ' + str(min_text_len))
print('Avg text length = ' + str(np.mean(all_lens)))
        